In [ ]:
pwd

In [ ]:
cd signature_verification

In [ ]:
ls dataset2/1/

In [ ]:
import sys
import numpy as np
import pickle
import os
import matplotlib.pyplot as plt
%matplotlib inline

import cv2
import time
import itertools
import random

from sklearn.utils import shuffle

import tensorflow as tf
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout, AveragePooling2D
from keras.models import Model

from tensorflow.keras.layers import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform

from tensorflow.keras.layers import Layer 
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.image import resize
from tensorflow.keras.callbacks import TensorBoard


In [ ]:
path = "dataset2/"


# Get the list of all directories and sort them
dir_list = next(os.walk(path))[1]
dir_list.sort()

In [ ]:
path = "dataset2/"


# Get the list of all directories and sort them
dir_list = next(os.walk(path))[1]
dir_list.sort()


orig_groups, forg_groups = [], []

for directory in dir_list:
    images = next(os.walk(path+directory))[2]
                 
    images.sort()
    images = [ path + directory+'/'+ x for x in images]
    forg_groups.append(images[:24]) # First 30 signatures in each folder are forrged
    orig_groups.append(images[24:])
    
    
    
# Quick check to confirm we have data of all the 160 individuals
len(orig_groups), len(forg_groups)



orig_lengths = [len(x) for x in orig_groups]
forg_lengths = [len(x) for x in forg_groups]



# Quick check to confirm that there are 24 Genuine signatures for each individual
print(orig_lengths)
print(forg_lengths)

orig_train, orig_val, orig_test = orig_groups[:40], orig_groups[40:50], orig_groups[50:]
forg_train, forg_val, forg_test = forg_groups[:40], forg_groups[40:50], forg_groups[50:]


img_h, img_w = 160, 160





In [ ]:
def visualize_sample_signature():
    '''Function to randomly select a signature from train set and
    print two genuine copies and one forged copy'''
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (10, 10))
    k = np.random.randint(len(orig_train))
    orig_img_names = random.sample(orig_train[k], 2)
    forg_img_name = random.sample(forg_train[k], 1)
    orig_img1 = cv2.imread(orig_img_names[0], 0)
    orig_img2 = cv2.imread(orig_img_names[1], 0)
    forg_img = plt.imread(forg_img_name[0], 0)
    orig_img1 = cv2.resize(orig_img1, (img_w, img_h))
    orig_img2 = cv2.resize(orig_img2, (img_w, img_h))
    forg_img = cv2.resize(forg_img, (img_w, img_h))

    ax1.imshow(orig_img1, cmap = 'gray')
    ax2.imshow(orig_img2, cmap = 'gray')
    ax3.imshow(forg_img, cmap = 'gray')

    ax1.set_title('Genuine Copy')
    ax1.axis('off')
    ax2.set_title('Genuine Copy')
    ax2.axis('off')
    ax3.set_title('Forged Copy')
    ax3.axis('off')
    
    
visualize_sample_signature()
visualize_sample_signature()
visualize_sample_signature()
visualize_sample_signature()

In [ ]:
forg_groups

In [ ]:
orig_pairs = []
forg_pairs = []
gen_gen_labels = []
gen_for_labels = []
all_pairs = []
all_labels = []

# Here we create pairs of Genuine-Genuine image names and Genuine-Forged image names
# For every person we have 24 genuine signatures, hence we have 
# 24 choose 2 = 276 Genuine-Genuine image pairs for one person.
# To make Genuine-Forged pairs, we pair every Genuine signature of a person
# with 12 randomly sampled Forged signatures of the same person.
# Thus we make 24 * 12 = 300 Genuine-Forged image pairs for one person.
# In all we have 120 person's data in the training data.
# Total no. of Genuine-Genuine pairs = 120 * 276 = 33120
# Total number of Genuine-Forged pairs = 120 * 300 = 36000
# Total no. of data points = 33120 + 36000 = 69120
for orig, forg in zip(orig_groups, forg_groups):

    orig_pairs.extend(list(itertools.combinations(orig, 2)))
    
    for i in range(len(forg)):
        forg_pairs.extend(list(itertools.product(orig[i:i+1], random.sample(forg, 12))))
        
print(orig_pairs)

In [ ]:
orig

In [ ]:
forg

In [ ]:

path = "dataset/"
path2="dataset2/"
#options = tf.data.Options()
#options.experimental_distribute.auto_shard_policy = AutoShardPolicy.OFF
# Get the list of all directories and sort them
dir_list = next(os.walk(path))[1]
dir_list.sort()
dir_list2 = next(os.walk(path2))[1]
dir_list2.sort()

orig_groups, forg_groups = [], []

##4000 dir
for directory in dir_list:
   
    images = next(os.walk(path+directory))[2]
    images.sort()
    images = [ path + directory+'/'+ x for x in images]
    orig_groups.append(images[96:]) # First 30 signatures in each folder are forrged
    forg_groups.append(images[:96])
    
print(len(orig_groups))
print(len(forg_groups))
##55 dir
for directory2 in dir_list2:
    images = next(os.walk(path2+directory2))[2]
    images.sort()
    images = [ path2 + directory2+'/'+ x for x in images]
    forg_groups.append(images[:24]) # First 30 signatures in each folder are forrged
    orig_groups.append(images[24:48])

    
# Quick check to confirm we have data of all the 160 individuals
print(len(orig_groups))
print(len(forg_groups))


orig_lengths = [len(x) for x in orig_groups]
forg_lengths = [len(x) for x in forg_groups]



# Quick check to confirm that there are 24 Genuine signatures for each individual
print(len(orig_lengths))
print(len(forg_lengths))
orig_train, orig_val, orig_test = orig_groups[:55], orig_groups[55:75], orig_groups[75:80]
forg_train, forg_val, forg_test = forg_groups[:55], forg_groups[55:75], forg_groups[75:80]


img_h, img_w = 160,160

In [ ]:
orig_pairs = []
forg_pairs = []
gen_gen_labels = []
gen_for_labels = []
all_pairs = []
all_labels = []

# Here we create pairs of Genuine-Genuine image names and Genuine-Forged image names
# For every person we have 24 genuine signatures, hence we have 
# 24 choose 2 = 276 Genuine-Genuine image pairs for one person.
# To make Genuine-Forged pairs, we pair every Genuine signature of a person
# with 12 randomly sampled Forged signatures of the same person.
# Thus we make 24 * 12 = 300 Genuine-Forged image pairs for one person.
# In all we have 120 person's data in the training data.
# Total no. of Genuine-Genuine pairs = 120 * 276 = 33120
# Total number of Genuine-Forged pairs = 120 * 300 = 36000
# Total no. of data points = 33120 + 36000 = 69120
for orig, forg in zip(orig_groups, forg_groups):    
    orig_pairs.extend(list(itertools.combinations(orig, 2)))
    for i in orig:
        for j in forg:
            forg_pairs.append((i,j))
        
print(len(orig_pairs))
print("_____")
print(len(forg_pairs))
print(orig_pairs[:20])
# p=orig_pairs + forg_pairs
# print(p)
gen_gen_labels = [1]*len(orig_pairs)
gen_for_labels = [0]*len(forg_pairs)

# Concatenate all the pairs together along with their labels and shuffle them
all_pairs = orig_pairs + forg_pairs
all_labels = gen_gen_labels + gen_for_labels
# print(all_pairs)

# del orig_pairs, forg_pairs, gen_gen_labels, gen_for_labels
all_pairs, all_labels = shuffle(all_pairs, all_labels)

# for i in range(0,12000):
#     if all_labels[i]==1:
#         print(all_pairs[i])
#         print(all_labels[i])
      

# Note the lists above contain only the image names and
# actual images are loaded and yielded below in batches
# Below we prepare a batch of data points and yield the batch
# In each batch we load "batch_size" number of image pairs
# These images are then removed from the original set so that
# they are not added again in the next batch.
# batch_size = 256
# k = 0
# pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
# targets=np.zeros((batch_size,))
# for ix, pair in enumerate(all_pairs):
# #     img1 = cv2.imread(pair[0], 0)
# #     img2 = cv2.imread(pair[1], 0)  
#     print(pair[0])
#     print(pair[1])

In [ ]:
print(all_labels)

In [ ]:
generate_batch(orig_train, forg_train, 256)
                              

In [ ]:
orig_train[:10]

In [ ]:
forg_train[:10]

In [ ]:
orig_val[:10]

In [ ]:
forg_val[:10]

In [ ]:
orig_pairs = []
        forg_pairs = []
        gen_gen_labels = []
        gen_for_labels = []
        all_pairs = []
        all_labels = []
        
        # Here we create pairs of Genuine-Genuine image names and Genuine-Forged image names
        # For every person we have 24 genuine signatures, hence we have 
        # 24 choose 2 = 276 Genuine-Genuine image pairs for one person.
        # To make Genuine-Forged pairs, we pair every Genuine signature of a person
        # with 12 randomly sampled Forged signatures of the same person.
        # Thus we make 24 * 12 = 300 Genuine-Forged image pairs for one person.
        # In all we have 120 person's data in the training data.
        # Total no. of Genuine-Genuine pairs = 120 * 276 = 33120
        # Total number of Genuine-Forged pairs = 120 * 300 = 36000
        # Total no. of data points = 33120 + 36000 = 69120
        for orig, forg in zip(orig_groups, forg_groups):    
            orig_pairs.extend(list(itertools.combinations(orig, 2)))
            for i in orig:
                for j in forg:
                    forg_pairs.append((i,j))
        

        
        # Label for Genuine-Genuine pairs is 1
        # Label for Genuine-Forged pairs is 0
        gen_gen_labels = [1]*len(orig_pairs)
        gen_for_labels = [0]*len(forg_pairs)
        
        # Concatenate all the pairs together along with their labels and shuffle them
        all_pairs = orig_pairs + forg_pairs
        all_labels = gen_gen_labels + gen_for_labels
        del orig_pairs, forg_pairs, gen_gen_labels, gen_for_labels
        all_pairs, all_labels = shuffle(all_pairs, all_labels)
        
        # Note the lists above contain only the image names and
        # actual images are loaded and yielded below in batches
        # Below we prepare a batch of data points and yield the batch
        # In each batch we load "batch_size" number of image pairs
        # These images are then removed from the original set so that
        # they are not added again in the next batch.
            
        k = 0
        pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
        targets=np.zeros((batch_size,))
        for ix, pair in enumerate(all_pairs):
            img1 = cv2.imread(pair[0], 0)
            img2 = cv2.imread(pair[1], 0)
            img1 = cv2.resize(img1, (img_w, img_h))
            img2 = cv2.resize(img2, (img_w, img_h))
            
            ret1,thresh1 = cv2.threshold(img1,200,255,cv2.THRESH_BINARY)
            ret2,thresh2 = cv2.threshold(img2,200,255,cv2.THRESH_BINARY)
            
            img1 = np.array(thresh1, dtype = np.float64)
            img2 = np.array(thresh2, dtype = np.float64)
            
            img1 /= 255
            img2 /= 255
            
            img1 = img1[..., np.newaxis]
            img2 = img2[..., np.newaxis]
            pairs[0][k, :, :, :] = img1
            pairs[1][k, :, :, :] = img2
            
            #print(pair[1])
            targets[k] = all_labels[ix]
            #print(targets[k])
            #print("f{pair[0]}:{pair[1]}:{targets[k]}")
            k += 1
            if k == batch_size:
                yield pairs,targets
                print(pairs)
                print(targets)
                k = 0
                pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
                targets=np.zeros((batch_size,))

In [ ]:
zef=[]

def generate_batch(orig_groups, forg_groups, batch_size = 256):
    '''Function to generate a batch of data with batch_size number of data points
    Half of the data points will be Genuine-Genuine pairs and half will be Genuine-Forged pairs'''
    while True:
        orig_pairs = []
        forg_pairs = []
        gen_gen_labels = []
        gen_for_labels = []
        all_pairs = []
        all_labels = []
        
        # Here we create pairs of Genuine-Genuine image names and Genuine-Forged image names
        # For every person we have 24 genuine signatures, hence we have 
        # 24 choose 2 = 276 Genuine-Genuine image pairs for one person.
        # To make Genuine-Forged pairs, we pair every Genuine signature of a person
        # with 12 randomly sampled Forged signatures of the same person.
        # Thus we make 24 * 12 = 300 Genuine-Forged image pairs for one person.
        # In all we have 120 person's data in the training data.
        # Total no. of Genuine-Genuine pairs = 120 * 276 = 33120
        # Total number of Genuine-Forged pairs = 120 * 300 = 36000
        # Total no. of data points = 33120 + 36000 = 69120
        for orig, forg in zip(orig_groups, forg_groups):    
            orig_pairs.extend(list(itertools.combinations(orig, 2)))
            for i in orig:
                for j in forg:
                    forg_pairs.append((i,j))
        

        
        # Label for Genuine-Genuine pairs is 1
        # Label for Genuine-Forged pairs is 0
        gen_gen_labels = [1]*len(orig_pairs)
        gen_for_labels = [0]*len(forg_pairs)
        
        # Concatenate all the pairs together along with their labels and shuffle them
        all_pairs = orig_pairs + forg_pairs
        all_labels = gen_gen_labels + gen_for_labels
        del orig_pairs, forg_pairs, gen_gen_labels, gen_for_labels
        all_pairs, all_labels = shuffle(all_pairs, all_labels)
        
        # Note the lists above contain only the image names and
        # actual images are loaded and yielded below in batches
        # Below we prepare a batch of data points and yield the batch
        # In each batch we load "batch_size" number of image pairs
        # These images are then removed from the original set so that
        # they are not added again in the next batch.
            
        k = 0
        pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
        targets=np.zeros((batch_size,))
        for ix, pair in enumerate(all_pairs):
            img1 = cv2.imread(pair[0], 0)
            img2 = cv2.imread(pair[1], 0)
            img1 = cv2.resize(img1, (img_w, img_h))
            img2 = cv2.resize(img2, (img_w, img_h))
            
            ret1,thresh1 = cv2.threshold(img1,200,255,cv2.THRESH_BINARY)
            ret2,thresh2 = cv2.threshold(img2,200,255,cv2.THRESH_BINARY)
            
            img1 = np.array(thresh1, dtype = np.float64)
            img2 = np.array(thresh2, dtype = np.float64)
            
            img1 /= 255
            img2 /= 255
            
            img1 = img1[..., np.newaxis]
            img2 = img2[..., np.newaxis]
            pairs[0][k, :, :, :] = img1
            pairs[1][k, :, :, :] = img2
            zef=pairs[0]
            print(zef)
            
            #print(pair[1])
            targets[k] = all_labels[ix]
            k += 1
            if k == batch_size:
                # pairs,targets
                print(pairs)
                print(targets)
#                 print("-----------------------")
                k = 0
                pairs=[np.zeros((batch_size, img_h, img_w, 1)) for i in range(2)]
                targets=np.zeros((batch_size,))
                
                

                
# def euclidean_distance(vects):
#     '''Compute Euclidean Distance between two vectors'''
#     x, y = vects
#     return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))
def euclidean_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    sum_square = tf.math.reduce_sum(tf.math.square(x - y), axis=1, keepdims=True)
    return tf.math.sqrt(tf.math.maximum(sum_square, tf.keras.backend.epsilon()))

def manhattan_distance(vects):
    """Find the Euclidean distance between two vectors.

    Arguments:
        vects: List containing two tensors of same length.

    Returns:
        Tensor containing euclidean distance
        (as floating point value) between vectors.
    """

    x, y = vects
    m_sum = tf.math.reduce_sum(abs(x - y), axis=1, keepdims=True)
    return tf.math.maximum(m_sum, tf.keras.backend.epsilon())



def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)




def loss(margin=1):
    """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

    Arguments:
        margin: Integer, defines the baseline for distance for which pairs
                should be classified as dissimilar. - (default is 1).

    Returns:
        'constrastive_loss' function with data ('margin') attached.
    """

    # Contrastive loss = mean( (1-true_value) * square(prediction) +
    #                         true_value * square( max(margin-prediction, 0) ))
    def contrastive_loss(y_true, y_pred):
        """Calculates the constrastive loss.

        Arguments:
            y_true: List of labels, each label is of type float32.
            y_pred: List of predictions of same length as of y_true,
                    each label is of type float32.

        Returns:
            A tensor containing constrastive loss as floating point value.
        """

        square_pred = tf.math.square(y_pred)
        margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
        return tf.math.reduce_mean(
            (1 - y_true) * square_pred + (y_true) * margin_square
        )

    return contrastive_loss


def BinaryCrossEntropy(y_true, y_pred):
    y_pred = np.clip(y_pred, 1e-7, 1 - 1e-7)
    term_0 = (1-y_true) * np.log(1-y_pred + 1e-7)
    term_1 = y_true * np.log(y_pred + 1e-7)
    return -np.mean(term_0+term_1, axis=0)



# def loss1(margin=1):
#     """Provides 'constrastive_loss' an enclosing scope with variable 'margin'.

#     Arguments:
#         margin: Integer, defines the baseline for distance for which pairs
#                 should be classified as dissimilar. - (default is 1).

#     Returns:
#         'constrastive_loss' function with data ('margin') attached.
#     """

#     # Contrastive loss = mean( (1-true_value) * square(prediction) +
#     #                         true_value * square( max(margin-prediction, 0) ))
#     def BinaryCrossEntropy(y_true, y_pred):
#         """Calculates the constrastive loss.

#         Arguments:
#             y_true: List of labels, each label is of type float32.
#             y_pred: List of predictions of same length as of y_true,
#                     each label is of type float32.

#         Returns:
#             A tensor containing constrastive loss as floating point value.
#         """

#         square_pred = tf.math.square(y_pred)
#         margin_square = tf.math.square(tf.math.maximum(margin - (y_pred), 0))
#         return tf.math.reduce_mean(
#             (1 - y_true) * square_pred + (y_true) * margin_square
#         )

#     return contrastive_loss



input_shape=(img_h, img_w, 1)

In [ ]:
generate_batch(orig_val, forg_val, batch_size = 256)

In [ ]:
# def signet2021(inputs):
#     '''Siamese Network with inception network'''

    
#     l_11 = Conv2D(32, kernel_size=(7, 7), activation='relu', strides=1,kernel_initializer='glorot_uniform', data_format='channels_last', padding = 'valid')(inputs)
#     l_12 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     l_13 = MaxPooling2D((2,2), strides=(2, 2))
#     l_14 = ZeroPadding2D((2, 2), data_format='channels_last')
    
    
    
#     l_2 = MaxPooling2D((3,3), strides=(2, 2))
    
    
#     l_31 = Conv2D(32, kernel_size=(1, 1), activation='relu', strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     l_32 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     l_33 = MaxPooling2D((2,2), strides=(2, 2))
#     l_34 = ZeroPadding2D((2, 2), data_format='channels_last')
    
    
    
    
    
#     l_41 = Conv2D(32, kernel_size=(3, 3), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     l_42 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     l_43 = MaxPooling2D((2,2), strides=(2, 2))
#     l_44 = ZeroPadding2D((2, 2), data_format='channels_last')
    
    
#     l_5 = MaxPooling2D((3,3), strides=(2, 2))
    
    
    
#     y_ = l_5(l_44(l_43(l_42(l_41(l_34(l_33(l_32(l_31(l_2(l_14(l_13(l_12(l_11)))))))))))))
    
#     x = y_
    
    
#     ######### Create Inception Network starts ##########
    
#     ### kernal_size = (1,1)
#     layer_1 = Conv2D(32, kernel_size=(7, 7), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     layer_12 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     layer_13 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
#     y_1 = layer_13(layer_12(layer_1(x)))
    


    
    
#     layer_21_1 = Conv2D(32, kernel_size=(1, 1), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     layer_22_1 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     layer_23_1 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    
    
#     layer_21_2 = Conv2D(32, kernel_size=(3, 3), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     layer_22_2 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     layer_23_2 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
      
#     y_2 = layer_23_2(layer_22_2(layer_21_2(layer_23_1(layer_22_1(layer_21_1(x))))))
    
    
    

    
#     layer_31_1 = Conv2D(32, kernel_size=(1, 1), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     layer_32_1 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     layer_33_1 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    
#     ### kernal_size = (5,5)
#     layer_31_2 = Conv2D(32, kernel_size=(3, 3), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     layer_32_2 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     layer_33_2 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    
#     y_3 = layer_33_2(layer_32_2(layer_31_2(layer_33_1(layer_32_1(layer_31_1(x))))))

    
    
#     layer_41 = MaxPooling2D((3,3), strides=(2, 2), padding='same')
#     ### kernal_size = (5,5)
#     layer_42 = Conv2D(32, kernel_size=(3, 3), activation='relu', strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
#     layer_43 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
#     layer_44 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    
#     y_4 = layer_44(layer_43(layer_42(layer_41(x))))

    
    
    
#     concatted = tf.keras.layers.Concatenate()([y_1, y_2, y_3, y_4])
    
    
#     #### inception layer ends ########
    
    
#     l_31 = AveragePooling2D(pool_size=(1, 1),strides=(2, 2), padding='valid')
    
#     l_32 = Dense(500, bias_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform')
    
    
#     y = l_32(l_31(concatted))
    
#     return y

In [ ]:
def signet2021(inputs):
    '''Siamese Network with inception network'''

    
    l_11 = Conv2D(32, kernel_size=(7, 7), activation='relu', strides=1,kernel_initializer='glorot_uniform', data_format='channels_last', padding = 'valid')(inputs)
    l_12 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    l_13 = MaxPooling2D((2,2), strides=(2, 2))
    l_14 = ZeroPadding2D((2, 2), data_format='channels_last')
    
    
    
    l_2 = MaxPooling2D((3,3), strides=(2, 2))
    
    
    l_31 = Conv2D(32, kernel_size=(1, 1), activation='relu', strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    l_32 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    l_33 = MaxPooling2D((2,2), strides=(2, 2))
    l_34 = ZeroPadding2D((2, 2), data_format='channels_last')
    
    
    
    
    
    l_41 = Conv2D(32, kernel_size=(3, 3), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    l_42 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    l_43 = MaxPooling2D((2,2), strides=(2, 2))
    l_44 = ZeroPadding2D((2, 2), data_format='channels_last')
    
    
    l_5 = MaxPooling2D((3,3), strides=(2, 2))
    
    
    
    y_ = l_5(l_44(l_43(l_42(l_41(l_34(l_33(l_32(l_31(l_2(l_14(l_13(l_12(l_11)))))))))))))
    
    x = y_
    
    
    ######### Create Inception Network starts ##########
    
    ### kernal_size = (1,1)
    layer_1 = Conv2D(32, kernel_size=(1, 1), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    layer_12 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    layer_13 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    y_1 = layer_13(layer_12(layer_1(x)))
    y_1 = resize(y_1, [4,4], method='bilinear')

    
    
    layer_21_1 = Conv2D(32, kernel_size=(1, 1), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    layer_22_1 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    layer_23_1 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    
    
    layer_21_2 = Conv2D(32, kernel_size=(3, 3), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    layer_22_2 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    layer_23_2 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
      
    y_2 = layer_23_2(layer_22_2(layer_21_2(layer_23_1(layer_22_1(layer_21_1(x))))))
    y_2 = resize(y_2, [4,4], method='bilinear')
    
    

    
    layer_31_1 = Conv2D(32, kernel_size=(1, 1), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    layer_32_1 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    layer_33_1 = MaxPooling2D((2,2), strides=(1, 1), padding='same')
    
    ### kernal_size = (5,5)
    layer_31_2 = Conv2D(32, kernel_size=(5, 5), activation='relu',  strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    layer_32_2 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    layer_33_2 = MaxPooling2D((2,2), strides=(2, 2), padding='same')
    
    y_3 = layer_33_2(layer_32_2(layer_31_2(layer_33_1(layer_32_1(layer_31_1(x))))))
    y_3 = resize(y_3, [4,4], method='bilinear')

    
    
    layer_41 = MaxPooling2D((3,3), strides=(1, 1), padding='same')
    ### kernal_size = (5,5)
    layer_42 = Conv2D(32, kernel_size=(5, 5), activation='relu', strides=1, kernel_initializer='glorot_uniform', data_format='channels_last',padding = 'valid')
    layer_43 = BatchNormalization(epsilon=1e-06, axis=1, momentum=0.9)
    layer_44 = MaxPooling2D((2,2), strides=(1,1), padding='same')
    
    y_4 = layer_44(layer_43(layer_42(layer_41(x))))
    y_4 = resize(y_4, [4,4], method='bilinear')

    
    
    
    concatted = tf.keras.layers.Concatenate()([ y_1, y_2, y_3, y_4 ])
    
    
    #### inception layer ends ########
    
    
    l_31 = AveragePooling2D(pool_size=(1, 1),strides=(2, 2), padding='valid')
    
    l_32 = Dense(500, bias_regularizer=l2(0.0005), activation='relu', kernel_initializer='glorot_uniform')
    
    
    y = l_32(l_31(concatted))
    
    return y







# network definition
# base_network = signet2021(inputs)

input_a = Input(shape=input_shape)
input_b = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = signet2021(input_a)
processed_b = signet2021(input_b)
print(processed_a)
print(processed_b)
# Compute the Euclidean distance between the two vectors in the latent space
# from sklearn.metrics.pairwise import manhattan_distances
# distance = manhattan_distances([processed_a], [processed_b])

# distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([processed_a, processed_b])
# print(distance)


merge_layer = Lambda(manhattan_distance)([processed_a, processed_b])
normal_layer = BatchNormalization()(merge_layer)
output_layer = Dense(1, activation="sigmoid")(normal_layer)





margin = 1
model = Model([input_a, input_b], output_layer)
model.summary()

# model = Model([input_a, input_b],distance)

In [ ]:
batch_sz = 256
num_train_samples = 276*40 + 576*40
num_val_samples = 276*10 + 576*10
num_test_samples = 276*5 + 576*5

# devices = tf.config.experimental.list_physical_devices('GPU')
# print(devices[0])
# # n_gpus=1

# # device_type = ‘GPU’

# devices = tf.config.experimental.list_physical_devices(
#           device_type)

# devices_names = [d.name.split(“e:”)[1] for d in devices]

# strategy = tf.distribute.MirroredStrategy(
#            devices=devices_names[:n_gpus])

# with strategy.scope():
#     model = Model([input_a, input_b], output_layer)
#     tb_callback = [
#     EarlyStopping(patience=12, verbose=1),
#     ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.0002, verbose=1),
#     ModelCheckpoint('./Weights/signet2021-bhsig260-{epoch:03d}.h5', verbose=1, save_weights_only=True)
#     ]


#     # compile model using RMSProp Optimizer and Contrastive loss function defined above
#     rms = RMSprop(learning_rate=0.0002, rho=0.9, epsilon=1e-08)
#     model.compile(loss=loss(margin=margin),optimizer=rms,  metrics=["accuracy"])


    

In [ ]:
!pip install tensorflow-gpu

In [ ]:
gpu_available = tf.test.is_gpu_available()
print(gpu_available)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

In [ ]:
# margin = 1
model.compile(loss=loss(margin=margin), optimizer="RMSprop", metrics=["accuracy"])

# num_train_samples, num_val_samples, num_test_samples
# model.summary()


# Using Keras Callbacks, save the model after every epoch
# Reduce the learning rate by a factor of 0.1 if the validation loss does not improve for 5 epochs
# Stop the training using early stopping if the validation loss does not improve for 12 epochs

root_logdir = os.path.join(os.curdir,"my_logs")

def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

# model call backs

tb_callback = TensorBoard(log_dir="/Users/jitendra/signature_verification/model_callback", histogram_freq=0, write_graph=True, write_images=True)

#model training
results = model.fit_generator(generate_batch(orig_train, forg_train, batch_sz),
                              steps_per_epoch = num_train_samples//batch_sz,
                              epochs = 1,
                              validation_data = generate_batch(orig_val, forg_val, batch_sz),
                              validation_steps = num_val_samples//batch_sz,
                              callbacks = [tb_callback, tensorboard_cb])



# save model and its weights
# model.save("Weights/signature_verification_model21")
# model.save_weights('Weights/signature_verification_model21.h5')


def plt_metric(results, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(results[metric])
    if has_valid:
        plt.plot(results["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(history=hresults.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(history=results.history, metric="loss", title="Constrastive Loss")

In [ ]:

def plt_metric(results, metric, title, has_valid=True):
    """Plots the given 'metric' from 'history'.

    Arguments:
        history: history attribute of History object returned from Model.fit.
        metric: Metric to plot, a string value present as key in 'history'.
        title: A string to be used as title of plot.
        has_valid: Boolean, true if valid data was passed to Model.fit else false.

    Returns:
        None.
    """
    plt.plot(results[metric])
    if has_valid:
        plt.plot(results["val_" + metric])
        plt.legend(["train", "validation"], loc="upper left")
    plt.title(title)
    plt.ylabel(metric)
    plt.xlabel("epoch")
    plt.show()


# Plot the accuracy
plt_metric(results.history, metric="accuracy", title="Model accuracy")

# Plot the constrastive loss
plt_metric(results.history, metric="loss", title="Constrastive Loss")

In [ ]:
def compute_accuracy_roc(predictions, labels):
    '''Compute ROC accuracy with a range of thresholds on distances.
    '''
    dmax = np.max(predictions)
    dmin = np.min(predictions)
    nsame = np.sum(labels == 1)
    ndiff = np.sum(labels == 0)
   
    step = 0.01
    max_acc = 0
    best_thresh = -1
   
    for d in np.arange(dmin, dmax+step, step):
        idx1 = predictions.ravel() <= d
        idx2 = predictions.ravel() > d
       
        tpr = float(np.sum(labels[idx1] == 1)) / nsame       
        tnr = float(np.sum(labels[idx2] == 0)) / ndiff
        acc = 0.5 * (tpr + tnr)       
#       print ('ROC', acc, tpr, tnr)
       
        if (acc > max_acc):
            max_acc, best_thresh = acc, d
           
    return max_acc, best_thresh

In [ ]:
!pip freeze > requirements.txt

In [ ]:
ls

In [ ]:
!nano requirements.txt


In [ ]:
test_gen = generate_batch(orig_test, forg_test, 1)
pred, tr_y = [], []
for i in range(num_test_samples):
    (img1, img2), label = next(test_gen)
    tr_y.append(label)
    pred.append(model.predict([img1, img2])[0][0])

In [ ]:
tr_acc, threshold = compute_accuracy_roc(np.array(pred), np.array(tr_y))
tr_acc, threshold

In [ ]:
def predict_score():
    '''Predict distance score and classify test images as Genuine or Forged'''
    test_point, test_label = next(test_gen)
    img1, img2 = test_point[0], test_point[1]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (10, 10))
    ax1.imshow(np.squeeze(img1), cmap='gray')
    ax2.imshow(np.squeeze(img2), cmap='gray')
    ax1.set_title('Genuine')
    if test_label == 1:
        ax2.set_title('Genuine')
    else:
        ax2.set_title('Forged')
    ax1.axis('off')
    ax2.axis('off')
    plt.show()
    result = model.predict([img1, img2])
    diff = result[0][0]
    print("Difference Score = ", diff)
    if diff > threshold:
        print("Its a Forged Signature")
    else:
        print("Its a Genuine Signature")

In [ ]:
predict_score()

In [ ]:
predict_score()

In [ ]:
predict_score()